https://github.com/langchain-ai/langchain/blob/master/templates/gemini-functions-agent/gemini_functions_agent/agent.py

In [22]:
import vertexai
vertexai.init(project="smart-agent-414407")

In [37]:
from typing import List, Tuple

from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import BaseTool, StructuredTool, tool

__all__ = ["agent_executor"]

In [38]:
from dotenv import load_dotenv
load_dotenv()  

True

In [39]:
from vectorengine import VectorEngine
ecsop = VectorEngine(folder_path="docs/general-docs", collection_name="sop")

UniqueConstraintError: Collection sop already exists

In [40]:
@tool
def search(query: str, max_results: int) -> str:
    """"A search engine optimized for comprehensive, accurate, \
    and trusted results. Useful for when you need to answer questions \
    about tennis, countries and trade operations. \
    Iteratively search if answer is not obtained. \
    Input should be a search query. \
    If the user is asking about something that you don't know about, \
    you should probably use this tool to see if that can provide any information."""
    result = ecsop.query(query, max_results)
    return result

In [41]:

# Create the tool
# description = """"A search engine optimized for comprehensive, accurate, \
# and trusted results. Useful for when you need to answer questions \
# about tennis, countries and trade operations. \
# Input should be a search query. \
# If the user is asking about something that you don't know about, \
# you should probably use this tool to see if that can provide any information."""
#tavily_tool = TavilySearchResults(api_wrapper=search, description=description)

tools = [search]

llm = ChatGoogleGenerativeAI(temperature=0, model="gemini-pro")

prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

llm_with_tools = llm.bind(functions=tools)


def _format_chat_history(chat_history: List[Tuple[str, str]]):
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer


agent = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: _format_chat_history(x["chat_history"]),
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)


class AgentInput(BaseModel):
    input: str
    chat_history: List[Tuple[str, str]] = Field(
        ..., extra={"widget": {"type": "chat", "input": "input", "output": "output"}}
    )


agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True).with_types(
    input_type=AgentInput
)

In [42]:
question = "what's the tourism webiste of the place where tennis player polanski was born?"
print(agent_executor.invoke({"input": question, "chat_history": []}))  # noqa: T201



> Entering new AgentExecutor chain...

Invoking: `search` with `{'query': 'tourism website of the place where tennis player polanski was born', 'max_results': 1.0}`


title:
polanski

content:
# Polanski: Serbia's Rising Tennis Star

![Polanski in action](URL_to_an_image_of_Polanski_playing_tennis)

## Early Life

Born in the heart of Serbia, Polanski discovered his passion for tennis at a young age. Hailing from a country known for producing some of the world's finest tennis talents, he was introduced to the sport at the tender age of four. Under the guidance of renowned local coaches, Polanski quickly distinguished himself with his natural talent and unwavering dedication.

## Career Beginnings

Polanski turned professional at the age of 16, following an impressive junior career that saw him clinch numerous national and international titles. His professional debut was marked by a stunning upset against a top-ranked player, signaling the arrival of a new force in tennis.

## Playing